# Programming Homework 4

### Question 1: In a practical, we saw the `scs function` (copied below along with \verb|overlap|overlap) for finding the shortest common superstring of a set of strings.

In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings `ABC`, `BCA`, `CAB`. One shortest common superstring is `ABCAB` but another is `BCABC` and another is `CABCA`.

What is the length of the shortest common superstring of the following strings?

`CCT`, `CTT`, `TGC`, `TGG`, `GAT`, `ATT`

In [2]:
def scs_list(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    shortest_sup_list = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) <= len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
            shortest_sup_list.append(sup)
    
    min_len = len(shortest_sup)
    out = list(filter(lambda x : len(x) == min_len, shortest_sup_list))
    out.sort()
    return out # return shortest

In [3]:
strings = ['ABC', 'BCA', 'CAB']
scs(strings)

'ABCAB'

In [4]:
scs_list(strings)

['ABCAB', 'BCABC', 'CABCA']

In [5]:
strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']
scs(strings)

'TCGATGCAATAG'

In [54]:
scs_list(strings)

['AATAGATCGTGC',
 'AATAGATGCTCG',
 'AATAGTCGATGC',
 'AATCGATAGTGC',
 'AATGCTCGATAG',
 'TCGAATAGATGC',
 'TCGATAGAATGC',
 'TCGATGCAATAG',
 'TGCAATAGATCG',
 'TGCAATCGATAG']

In [6]:
strings = ["CCT", 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
scs(strings)

'CCTTGGATTGC'

In [57]:
len(scs(strings))

11

In [7]:
scs_list(strings)

['CCTTGGATTGC', 'GATTGCCTTGG', 'TGCCTTGGATT', 'TGGATTGCCTT']

In [8]:
len(scs_list(strings))

4

In [9]:
import wget
import os

url = 'https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq'

fname = os.path.basename(url)

if not os.path.isfile('data/'+fname):
    wget.download(url, out='data/')
    print(fname + " File Downloaded...")
else: 
    print(fname + " Already Exist! ")

ads1_week4_reads.fq Already Exist! 


In [10]:
def pick_max_overlap(reads, k):
    best_a, best_b, best_len = None, None, 0
    for a, b in itertools.permutations(reads, 2):
        length = overlap(a, b, min_length=k)
        if length > best_len:
            best_a, best_b, best_len = a, b, length
    return best_a, best_b, best_len

In [11]:
def greedy_scss(reads, k):
    while True:
        a, b, olen = pick_max_overlap(reads, k)
        if olen == 0:
            break
        reads.remove(a)
        reads.remove(b)
        reads.append(a + b[olen:])
    return ''.join(reads) # append all non-overlaps onto eachother and return the concatenated string

In [12]:
greedy_scss(['ABC', 'BCA', 'CAB'], 2)

'CABCA'

In [13]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0: # If line reaches end of file seq value become 0
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [14]:
mystery_virus, _ = readFastq('data/'+fname)


In [15]:
genome = greedy_scss(mystery_virus, 10)

KeyboardInterrupt: 

In [16]:
import operator


def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


def smart_overlap_map(reads, k):
    olaps = {}
    result = {}
    for read in reads:
        for i in range(len(read)-k+1):
            if read[i:i+k] not in olaps:
                olaps[read[i:i+k]] = [read]
            else:
                olaps[read[i:i+k]].append(read)

    count = 0
    for read in reads:
        read_suffix = read[-k:]
        for possible_read in olaps[read_suffix]:
            if possible_read != read:
                olen = overlap(read, possible_read, k)
                if olen > 0:
                    count += 1
                    result[(read, possible_read)] = olen

    return result, count


def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen


def smart_greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    pairs_olen, pairs_count = smart_overlap_map(reads, k)
    sorted_pairs_olen = sorted(pairs_olen.items(), key=operator.itemgetter(1), reverse=True)
    read_a, read_b, olen = sorted_pairs_olen[0][0][0], sorted_pairs_olen[0][0][1], sorted_pairs_olen[0][1]
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        pairs_olen, pairs_count = smart_overlap_map(reads, k)
        if pairs_olen != {}:
            sorted_pairs_olen = sorted(pairs_olen.items(), key=operator.itemgetter(1), reverse=True)
            read_a, read_b, olen = sorted_pairs_olen[0][0][0], sorted_pairs_olen[0][0][1], sorted_pairs_olen[0][1]
        else:
            read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [17]:
genome = smart_greedy_scs(mystery_virus, 10)
print("Number of As in the assembled genome:", genome.count('A'))
print("Number of Ts in the assembled genome:", genome.count('T'))

Number of As in the assembled genome: 4633
Number of Ts in the assembled genome: 3723


In [24]:
out = "\n".join(['> mystery_virus|' + str(len(genome)),genome])
with open('out/mystery_virus_genome.txt', 'w') as output_data:
    output_data.write(out)